In [ ]:
import pandas as pd
import re
from textblob import TextBlob
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset, specifying the engine
df = pd.read_excel(r"C:\Users\Lenovo\Downloads\redmi6.xls", engine='xlrd')

# Data Cleaning
def clean_comment(comment):
    comment = re.sub(r"http\S+|www\S+|@\w+|#\w+", '', comment)  # Remove URLs, mentions, hashtags
    comment = re.sub(r'[^A-Za-z\s]', '', comment)  # Remove non-alphabetic characters
    return str(TextBlob(comment).correct())

df['Comment'] = df['Comment'].apply(clean_comment)

# Drop rows with missing essential data (e.g., Rating or Comment)
df.dropna(subset=['Rating', 'Comment'], inplace=True)

# Sentiment Analysis
def get_sentiment(comment):
    analysis = TextBlob(comment)
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity == 0:
        return 'Neutral'
    else:
        return 'Negative'

df['Sentiment'] = df['Comment'].apply(get_sentiment)

# Trend Analysis
df['Date'] = pd.to_datetime(df['Date'])
trend_df = df.groupby('Date').agg({'Sentiment': 'value_counts', 'Rating': 'mean'}).unstack()
channel_df = df.groupby('Feedback Channel').agg({
    'Rating': 'mean',
    'Sentiment': lambda x: x.value_counts().to_dict()
})

# Data Visualization
sns.countplot(data=df, x='Sentiment')
plt.title("Sentiment Distribution")
plt.show()

channel_avg_rating = df.groupby('Feedback Channel')['Rating'].mean()
channel_avg_rating.plot(kind='bar')
plt.title("Average Rating by Feedback Channel")
plt.show()

df.groupby('Date')['Sentiment'].value_counts().unstack().plot(kind='line')
plt.title("Feedback Trends Over Time")
plt.show()
